In [2]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [10]:
#MODEL
list_i = []
list_return_price = []
list_rmse = []
list_mae =[]
list_mse = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    model = LinearRegression()
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {mse:.2f}',f' RMSE: {mse:.2f}', f'MAE: {mae:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -2.64  R2_train: 0.59  R2_test: 0.45  RMSE: 2.57  RMSE: 2.57 MAE: 1.36
Stock: BCM  Predict: -1.24  R2_train: 0.46  R2_test: -0.51  RMSE: 1.77  RMSE: 1.77 MAE: 1.13
Stock: VHM  Predict: 1.75  R2_train: 0.51  R2_test: 0.52  RMSE: 2.38  RMSE: 2.38 MAE: 1.12
Stock: VIC  Predict: 1.23  R2_train: 0.50  R2_test: 0.31  RMSE: 2.89  RMSE: 2.89 MAE: 1.18
Stock: VRE  Predict: -1.66  R2_train: 0.56  R2_test: 0.46  RMSE: 1.75  RMSE: 1.75 MAE: 1.09
Stock: BVH  Predict: 1.42  R2_train: 0.58  R2_test: 0.18  RMSE: 1.15  RMSE: 1.15 MAE: 0.93
Stock: POW  Predict: 1.72  R2_train: 0.58  R2_test: 0.38  RMSE: 1.72  RMSE: 1.72 MAE: 1.07
Stock: GAS  Predict: 1.51  R2_train: 0.56  R2_test: 0.02  RMSE: 1.12  RMSE: 1.12 MAE: 0.93
Stock: ACB  Predict: 1.30  R2_train: 0.53  R2_test: 0.27  RMSE: 1.20  RMSE: 1.20 MAE: 0.95
Stock: BID  Predict: 1.58  R2_train: 0.55  R2_test: 0.41  RMSE: 1.61  RMSE: 1.61 MAE: 1.03
Stock: CTG  Predict: 1.77  R2_train: 0.57  R2_test: 0.39  RMSE: 1.52  RMSE: 1.52 MAE: 

In [11]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test, 'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,})
KQ.to_excel("Kết quả 20% test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE
0,SSI,[-2.641332092891991],0.594288,0.453638,2.566851,1.602140,1.358905
1,BCM,[-1.2353036351439375],0.459907,-0.511202,1.773578,1.331757,1.130084
2,VHM,[1.7508604099087555],0.514318,0.520568,2.376093,1.541458,1.120746
3,VIC,[1.2325110884585486],0.498002,0.311288,2.893980,1.701170,1.180663
4,VRE,[-1.6588504162072122],0.555954,0.455462,1.749571,1.322714,1.087485
...,...,...,...,...,...,...,...
389,TDP,[0.26758375671960927],0.363612,0.242170,4.503512,2.122148,1.838271
390,NO1,[-1.6184958525234494],0.605308,0.325792,7.875338,2.806303,2.174915
391,L10,[1.441449182076769],0.649726,0.576620,15.340100,3.916644,2.923503
392,REE,[1.448241346784749],0.517482,0.264263,1.337540,1.156521,1.006285
